In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot = True)

# 批次
batch_size = 100
# 多少个批次
n_batch = mnist.train.num_examples // batch_size

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)#标准差
        tf.summary.scalar('max', tf.reduce_max(var))#最大值
        tf.summary.scalar('min', tf.reduce_min(var))#最小值
        tf.summary.histogram('histogram', var)#直方图

# weight,bias,conv2d,max_pooling
def Weight_variable(shape,name):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial,name=name)

def bias_variable(shape,name):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial,name=name)

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

# placeholder
with tf.name_scope('input') as scope:
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')
#     lr = tf.Variable(0.001,dtype=tf.float32)
    keep_prob = tf.placeholder(tf.float32,name='keep_prob')

# 输入的x
with tf.name_scope('x_image') as scope:
    x_image = tf.reshape(x,[-1,28,28,1],name='x_image')

# conv1
with tf.name_scope('Conv1') as scope:
    with tf.name_scope('W_conv1') as scope:
        W_conv1 = Weight_variable([5,5,1,32],name=('W_conv1'))
    with tf.name_scope('b_conv1') as scope:
        b_conv1 = bias_variable([32],name='b_conv1')
    with tf.name_scope('h_conv1') as scope:
        h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)
    with tf.name_scope('h_max_pool1'):
        h_max_pool1 = max_pool_2x2(h_conv1,)

# conv2
with tf.name_scope('Conv2'):
    with tf.name_scope('W_conv2'):
        W_conv2 = Weight_variable([5,5,32,64],name='W_conv2')
    with tf.name_scope('b_conv2'):
        b_conv2 = bias_variable([64],name=('b_conv2'))
    with tf.name_scope('h_conv2'):
        h_conv2 = tf.nn.relu(conv2d(h_max_pool1,W_conv2) + b_conv2)
    with tf.name_scope('h_max_pool2'):
        h_max_pool2 = max_pool_2x2(h_conv2)

# 经过两层卷积和池化,变成了一张7*7*64的feature map
with tf.name_scope('h_pool2_flat'):
    h_pool2_flat = tf.reshape(h_max_pool2,[-1,7*7*64])

# fc1
with tf.name_scope('fc1'):
    with tf.name_scope('W_fc1'):
        W_fc1 = Weight_variable([7*7*64,1024],name='W_fc1')
    with tf.name_scope('b_fc1'):
        b_fc1 = bias_variable([1024],name='b_fc1')
    with tf.name_scope('h_fc1'):
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)
    with tf.name_scope('h_fc1_drop'):
        h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

# fc2 ----> prediction
with tf.name_scope('fc2'):
    with tf.name_scope('W_fc2'):
        W_fc2 = Weight_variable([1024,10],name='W_fc2')
    with tf.name_scope('b_fc2'):
        b_fc2 = bias_variable([10],name='b_fc2')
    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)

# 交叉熵损失函数以及优化器
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('cross_entropy',cross_entropy)
    
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# accuracy
with tf.name_scope('accuracy'):
    with tf.name_scope('coorect_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float32"))
        tf.summary.scalar('accuracy',accuracy)
        
# initial
init = tf.global_variables_initializer()

merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter('logs/Train',sess.graph)
    test_writer = tf.summary.FileWriter('logs/Test',sess.graph)
    
    for i in range(1001):
        batch = mnist.train.next_batch(batch_size)
        sess.run(train_step,feed_dict={x:batch[0],y:batch[1],keep_prob:0.5})
        
        #记录训练集计算的参数
        summary = sess.run(merged,feed_dict={x:batch[0],y:batch[1],keep_prob:1.0})
        # 这里不懂
        train_writer.add_summary(summary,i)
        
        batch2 = mnist.test.next_batch(batch_size)
        summary2 = sess.run(merged,feed_dict={x:batch2[0],y:batch2[1],keep_prob:1.0})
        test_writer.add_summary(summary2,i)
        
        if i % 100 == 0:
            test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
            train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images[:10000],y:mnist.train.labels[:10000],keep_prob:1.0})
            print("Iter" + str(i) + " Test accuracy: " + str(test_acc) + " Train accuracy: " + str(train_acc))           
                                              
                                              
                                              
                                              
# 	for epoch in range(11):
# # 		sess.run(tf.assign(lr,0.001 * (0.95 ** epoch)))
# 		for batch in range(n_batch):
# 			batch = mnist.train.next_batch(batch_size)
# 			sess.run(train_step,feed_dict={x:batch[0],y:batch[1],keep_prob:0.7})

# # 		learning_rate = sess.run(lr)
# 		test_acc = sess.run(accuracy,feed_dict={x:batch[0],y:batch[1],keep_prob:1.0})
# 		print('Iter: ' + str(epoch) + ',Test_accuracy: ' + str(test_acc))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Iter0 Test accuracy: 0.1215 Train accuracy: 0.1189
Iter100 Test accuracy: 0.6093 Train accuracy: 0.6081
Iter200 Test accuracy: 0.7377 Train accuracy: 0.7363
Iter300 Test accuracy: 0.7716 Train accuracy: 0.77
Iter400 Test accuracy: 0.8546 Train accuracy: 0.8533
Iter500 Test accuracy: 0.8597 Train accuracy: 0.8598
Iter600 Test accuracy: 0.8629 Train accuracy: 0.861
Iter700 Test accuracy: 0.8681 Train accuracy: 0.8661
Iter800 Test accuracy: 0.9576 Train accuracy: 0.9526
Iter900 Test accuracy: 0.9613 Train accuracy: 0.9578
Iter1000 Test accuracy: 0.9647 Train accuracy: 0.9612
